In [9]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets,transforms

In [14]:

transform = transforms.Compose([
    transforms.Resize((128, 128)),  # Resize images to a fixed size
    transforms.ToTensor(),  # Convert images to PyTorch tensors
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize the images
])
train_dataset=datasets.ImageFolder("~/prjdata/train_and_test/train",transform=transform)
test_dataset=datasets.ImageFolder("~/prjdata/train_and_test/test",transform=transform)
print("Done")

Done


In [13]:
trainloader=DataLoader(train_dataset,batch_size=32)
testloader=DataLoader(test_dataset,batch_size=32)
print("Done")

Done


In [15]:
import torch.nn as nn
import torch.nn.functional as F

In [16]:
class LogoNet(nn.Module):
    def __init__(self):
        super(LogoNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(64 * 32 * 32, 512)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * 32 * 32)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
     
model = LogoNet()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)





In [6]:
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in trainloader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(trainloader):.4f}')

    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in testcodeloader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    print(f'Validation Accuracy: {100 * correct / total}%')

Epoch [1/2], Loss: 24.6685
Epoch [2/2], Loss: 4.1744
